# Working with Open Model Zoo Models

Objective:
- To download a model from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo), 
- convert it to OpenVINO™ IR format, 
- show information about the model, 
- and benchmark the model.

## OpenVINO and Open Model Zoo Tools

OpenVINO and Open Model Zoo tools are listed in the table below.

| Tool             | Command             | Description                                             |
|:-----------------|:--------------------|:--------------------------------------------------------|
| Model Downloader | omz_downloader      | Download models from Open Model Zoo.                    |
| Model Converter  | omz_converter       | Convert Open Model Zoo models to OpenVINO's IR format.  |
| Info Dumper      | omz_info_dumper     | Print information about Open Model Zoo models.          |
| Benchmark Tool   | benchmark_app       | Benchmark model performance by computing inference time.|

## Preparation

### Model Name

Set `model_name` to the name of the Open Model Zoo model to use in this notebook. Refer to the list of [public](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/index.md) and [Intel](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/index.md) pre-trained models for a full list of models that can be used. Set `model_name` to the model you want to use.

In [1]:
# model_name = "resnet-50-pytorch"
model_name = "mobilenet-v2-pytorch"

### Imports

In [2]:
import json
import sys
from pathlib import Path

from IPython.display import Markdown, display
from openvino.runtime import Core

sys.path.append("../utils")
from notebook_utils import DeviceNotFoundAlert, NotebookAlert

### Settings and Configuration

Set the file and directory paths. By default, this notebook downloads models from Open Model Zoo to the `open_model_zoo_models` directory in your `$HOME` directory. On Windows, the $HOME directory is usually `c:\users\username`, on Linux `/home/username`. To change the folder, change `base_model_dir` in the cell below.

The following settings can be changed:

* `base_model_dir`: Models will be downloaded into the `intel` and `public` folders in this directory.
* `omz_cache_dir`: Cache folder for Open Model Zoo. Specifying a cache directory is not required for Model Downloader and Model Converter, but it speeds up subsequent downloads.
* `precision`: If specified, only models with this precision will be downloaded and converted.

In [3]:
base_model_dir = Path("~/open_model_zoo_models").expanduser()
omz_cache_dir = Path("~/open_model_zoo_cache").expanduser()
precision = "FP16"

# Check if an iGPU is available on this system to use with Benchmark App.
ie = Core()
gpu_available = "GPU" in ie.available_devices

print(
    f"base_model_dir: {base_model_dir}, omz_cache_dir: {omz_cache_dir}, gpu_availble: {gpu_available}"
)

base_model_dir: C:\Users\97798\open_model_zoo_models, omz_cache_dir: C:\Users\97798\open_model_zoo_cache, gpu_availble: True


## Download a Model from Open Model Zoo



Specify, display and run the Model Downloader command to download the model.

In [5]:
## Uncomment the next line to show help in omz_downloader which explains the command-line options.

!omz_downloader --help

usage: omz_downloader [-h] [--name PAT[,PAT...]] [--list FILE.LST] [--all]
                      [--print_all] [--precisions PREC[,PREC...]] [-o DIR]
                      [--cache_dir DIR] [--num_attempts N]
                      [--progress_format {text,json}] [-j N]

optional arguments:
  -h, --help            show this help message and exit
  --name PAT[,PAT...]   download only models whose names match at least one of
                        the specified patterns
  --list FILE.LST       download only models whose names match at least one of
                        the patterns in the specified file
  --all                 download all available models
  --print_all           print all available models
  --precisions PREC[,PREC...]
                        download only models with the specified precisions
                        (actual for DLDT networks); specify one or more of:
                        FP32-INT8,FP16-INT1,FP32,FP16,FP32-INT1,FP16-INT8
  -o DIR, --output_dir DIR
  

In [6]:
download_command = (
    f"omz_downloader --name {model_name} --output_dir {base_model_dir} --cache_dir {omz_cache_dir}"
)
display(Markdown(f"Download command: `{download_command}`"))
display(Markdown(f"Downloading {model_name}..."))
! $download_command

Download command: `omz_downloader --name mobilenet-v2-pytorch --output_dir C:\Users\97798\open_model_zoo_models --cache_dir C:\Users\97798\open_model_zoo_cache`

Downloading mobilenet-v2-pytorch...

################|| Downloading mobilenet-v2-pytorch ||################

========== Downloading C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\mobilenet_v2-b0353104.pth
... 7%, 1024 KB, 1490 KB/s, 0 seconds passed
... 14%, 2048 KB, 2149 KB/s, 0 seconds passed
... 22%, 3072 KB, 2489 KB/s, 1 seconds passed
... 29%, 4096 KB, 2818 KB/s, 1 seconds passed
... 36%, 5120 KB, 3064 KB/s, 1 seconds passed
... 44%, 6144 KB, 3276 KB/s, 1 seconds passed
... 51%, 7168 KB, 3424 KB/s, 2 seconds passed
... 59%, 8192 KB, 3543 KB/s, 2 seconds passed
... 66%, 9216 KB, 3532 KB/s, 2 seconds passed
... 73%, 10240 KB, 3662 KB/s, 2 seconds passed
... 81%, 11264 KB, 3697 KB/s, 3 seconds passed
... 88%, 12288 KB, 3763 KB/s, 3 seconds passed
... 95%, 13312 KB, 3737 KB/s, 3 seconds passed
... 100%, 13879 KB, 3764 KB/s, 3 seconds passed



## Convert a Model to OpenVINO IR format

Specify, display and run the Model Converter command to convert the model to OpenVINO IR format. Model conversion may take a while. The output of the Model Converter command will be displayed. When the conversion is successful, the last lines of the output will include: `[ SUCCESS ] Generated IR version 11 model.` For downloaded models that are already in OpenVINO IR format, conversion will be skipped.

In [7]:
## Uncomment the next line to show Help in omz_converter which explains the command-line options.

# !omz_converter --help

In [8]:
convert_command = f"omz_converter --name {model_name} --precisions {precision} --download_dir {base_model_dir} --output_dir {base_model_dir}"
display(Markdown(f"Convert command: `{convert_command}`"))
display(Markdown(f"Converting {model_name}..."))

! $convert_command

Convert command: `omz_converter --name mobilenet-v2-pytorch --precisions FP16 --download_dir C:\Users\97798\open_model_zoo_models --output_dir C:\Users\97798\open_model_zoo_models`

Converting mobilenet-v2-pytorch...

========== Converting mobilenet-v2-pytorch to ONNX
Conversion to ONNX command: C:\Users\97798\openvino_en\Scripts\python.exe -- C:\Users\97798\openvino_en\lib\site-packages\openvino\model_zoo\internal_scripts\pytorch_to_onnx.py --model-name=mobilenet_v2 --weights=C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch/mobilenet_v2-b0353104.pth --import-module=torchvision.models --input-shape=1,3,224,224 --output-file=C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch/mobilenet-v2.onnx --input-names=data --output-names=prob

ONNX check passed successfully.

========== Converting mobilenet-v2-pytorch to IR (FP16)
Conversion command: C:\Users\97798\openvino_en\Scripts\python.exe -- C:\Users\97798\openvino_en\Scripts\mo.exe --framework=onnx --data_type=FP16 --output_dir=C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16 --model_name=mobilenet-v2-pytorch --input=data --mean_values=data[123.675,116.28,103.53] --scale_values=data[58.624,57.12,57.375] 

## Get Model Information

The Info Dumper prints the following information for Open Model Zoo models:

* Model name
* Description
* Framework that was used to train the model
* License URL
* Precisions supported by the model
* Subdirectory: the location of the downloaded model
* Task type

This information can be shown by running `omz_info_dumper --name model_name` in a terminal. The information can also be parsed and used in scripts. 

In the next cell, run Info Dumper and use `json` to load the information in a dictionary. 

In [9]:
model_info_output = %sx omz_info_dumper --name $model_name
model_info = json.loads(model_info_output.get_nlstr())

if len(model_info) > 1:
    NotebookAlert(
        f"There are multiple IR files for the {model_name} model. The first model in the "
        "omz_info_dumper output will be used for benchmarking. Change "
        "`selected_model_info` in the cell below to select a different model from the list.",
        "warning",
    )

model_info

[{'name': 'mobilenet-v2-pytorch',
  'composite_model_name': None,
  'description': 'MobileNet V2 is image classification model pre-trained on ImageNet dataset. This is a PyTorch* implementation of MobileNetV2 architecture as described in the paper "Inverted Residuals and Linear Bottlenecks: Mobile Networks for Classification, Detection and Segmentation" <https://arxiv.org/abs/1801.04381>.\nThe model input is a blob that consists of a single image of "1, 3, 224, 224" in "RGB" order.\nThe model output is typical object classifier for the 1000 different classifications matching with those in the ImageNet database.',
  'framework': 'pytorch',
  'license_url': 'https://raw.githubusercontent.com/pytorch/vision/master/LICENSE',
  'accuracy_config': 'C:\\Users\\97798\\openvino_en\\Lib\\site-packages\\openvino\\model_zoo\\models\\public\\mobilenet-v2-pytorch\\accuracy-check.yml',
  'model_config': 'C:\\Users\\97798\\openvino_en\\Lib\\site-packages\\openvino\\model_zoo\\models\\public\\mobilenet

Having information of the model in a JSON file allows extracting the path to the model directory, and building the path to the OpenVINO IR file.

In [10]:
selected_model_info = model_info[0]
model_path = (
    base_model_dir
    / Path(selected_model_info["subdirectory"])
    / Path(f"{precision}/{selected_model_info['name']}.xml")
)
print(model_path, "exists:", model_path.exists())

C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16\mobilenet-v2-pytorch.xml exists: True


## Run Benchmark Tool

By default, Benchmark Tool runs inference for 60 seconds in asynchronous mode on CPU. It returns inference speed as latency (milliseconds per image) and throughput values (frames per second). 

In [11]:
## Uncomment the next line to show Help in benchmark_app which explains the command-line options.
# !benchmark_app --help

In [12]:
benchmark_command = f"benchmark_app -m {model_path} -t 15"
display(Markdown(f"Benchmark command: `{benchmark_command}`"))
display(Markdown(f"Benchmarking {model_name} on CPU with async inference for 15 seconds..."))

! $benchmark_command

Benchmark command: `benchmark_app -m C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16\mobilenet-v2-pytorch.xml -t 15`

Benchmarking mobilenet-v2-pytorch on CPU with async inference for 15 seconds...

[Step 1/11] Parsing and validating input arguments
[ WARNING ]  -nstreams default value is determined automatically for a device. Although the automatic selection usually provides a reasonable performance, but it still may be non-optimal for some cases, for more information look at README. 
[Step 2/11] Loading OpenVINO
[ WARNING ] PerformanceMode was not explicitly specified in command line. Device CPU performance hint will be set to THROUGHPUT.
[ INFO ] OpenVINO:
         API version............. 2022.2.0-7713-af16ea1d79a-releases/2022/2
[ INFO ] Device info
         CPU
         openvino_intel_cpu_plugin version 2022.2
         Build................... 2022.2.0-7713-af16ea1d79a-releases/2022/2

[Step 3/11] Setting device configuration
[ WARNING ] -nstreams default value is determined automatically for CPU device. Although the automatic selection usually provides a reasonable performance, but it still may be non-optimal for some cases, for more information look at README.
[Step 4/11] 

### Benchmark with Different Settings

The `benchmark_app` tool displays logging information that is not always necessary. A more compact result is achieved when the output is parsed with `json`.

The following cells show some examples of `benchmark_app` with different parameters. Below are some useful parameters:

- `-d` A device to use for inference. For example: CPU, GPU, MULTI. Default: CPU.
- `-t` Time expressed in number of seconds to run inference. Default: 60.
- `-api` Use asynchronous (async) or synchronous (sync) inference. Default: async.
- `-b` Batch size. Default: 1.


Run `! benchmark_app --help` to get an overview of all possible command-line parameters.

In the next cell, define the `benchmark_model()` function that calls `benchmark_app`. This makes it easy to try different combinations. In the cell below that, you display available devices on the system.

> **Note**: In this notebook, `benchmark_app` runs for 15 seconds to give a quick indication of performance. For more accurate performance, it is recommended to run inference for at least one minute by setting the `t` parameter to 60 or higher, and run `benchmark_app` in a terminal/command prompt after closing other applications. Copy the **benchmark command** and paste it in a command prompt where you have activated the `openvino_env` environment. 

In [13]:
def benchmark_model(model_xml, device="CPU", seconds=60, api="async", batch=1):
    ie = Core()
    model_path = Path(model_xml)
    if ("GPU" in device) and ("GPU" not in ie.available_devices):
        DeviceNotFoundAlert("GPU")
    else:
        benchmark_command = f"benchmark_app -m {model_path} -d {device} -t {seconds} -api {api} -b {batch}"
        display(Markdown(f"**Benchmark {model_path.name} with {device} for {seconds} seconds with {api} inference**"))
        display(Markdown(f"Benchmark command: `{benchmark_command}`"))

        benchmark_output = %sx $benchmark_command
        print("command ended")
        benchmark_result = [line for line in benchmark_output
                            if not (line.startswith(r"[") or line.startswith("      ") or line == "")]
        print("\n".join(benchmark_result))

In [14]:
ie = Core()

# Show devices available for OpenVINO Runtime
for device in ie.available_devices:
    device_name = ie.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU: Intel(R) Celeron(R) N4020 CPU @ 1.10GHz
GPU: Intel(R) UHD Graphics 600 (iGPU)


In [15]:
benchmark_model(model_path, device="CPU", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with CPU for 15 seconds with async inference**

Benchmark command: `benchmark_app -m C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16\mobilenet-v2-pytorch.xml -d CPU -t 15 -api async -b 1`

command ended
Count:          465 iterations
Duration:       15078.35 ms
Latency:
    Median:     29.49 ms
    AVG:        32.31 ms
    MIN:        27.57 ms
    MAX:        75.53 ms
Throughput: 30.84 FPS


In [16]:
benchmark_model(model_path, device="AUTO", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with AUTO for 15 seconds with async inference**

Benchmark command: `benchmark_app -m C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16\mobilenet-v2-pytorch.xml -d AUTO -t 15 -api async -b 1`

command ended
Count:          256 iterations
Duration:       18267.93 ms
Latency:
    Median:     1117.30 ms
    AVG:        2151.47 ms
    MIN:        164.79 ms
    MAX:        18211.60 ms
Throughput: 14.01 FPS


In [17]:
benchmark_model(model_path, device="GPU", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with GPU for 15 seconds with async inference**

Benchmark command: `benchmark_app -m C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16\mobilenet-v2-pytorch.xml -d GPU -t 15 -api async -b 1`

command ended
Count:          1552 iterations
Duration:       15241.57 ms
Latency:
    Median:     154.85 ms
    AVG:        156.36 ms
    MIN:        54.53 ms
    MAX:        219.48 ms
Throughput: 101.83 FPS


In [18]:
benchmark_model(model_path, device="MULTI:CPU,GPU", seconds=15, api="async")

**Benchmark mobilenet-v2-pytorch.xml with MULTI:CPU,GPU for 15 seconds with async inference**

Benchmark command: `benchmark_app -m C:\Users\97798\open_model_zoo_models\public\mobilenet-v2-pytorch\FP16\mobilenet-v2-pytorch.xml -d MULTI:CPU,GPU -t 15 -api async -b 1`

command ended
Count:          799 iterations
Duration:       15267.84 ms
Throughput: 52.33 FPS
